# Importing Libraries and Configurations

https://github.com/aws/amazon-sagemaker-examples/issues/1207


https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html

In [1]:
import os
import glob
import json
import boto3
import sagemaker


# Taking pipeline building configurations from config.json.
# These are only for building and will not be available at 
# the runtime of the pipeline.
with open("config.json") as file:
    build_parameters = json.load(file)

# Setting Default Bucket and getting region and role

In [2]:

## Setting default bucket
# Method 1
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession(default_bucket = build_parameters["output_bucket"])
sagemaker_session = sagemaker.Session(default_bucket = build_parameters["output_bucket"])

# Method 2
pipeline_session.default_bucket = build_parameters["output_bucket"]
sagemaker_session.default_bucket = build_parameters["output_bucket"]

# Method 3
# sagemaker_session = sagemaker.Session()
# bucket = "sagemaker-output-bucket-us-east1"   

print(sagemaker_session.default_bucket)
print(pipeline_session.default_bucket)


## Getting region and role
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

print(role)

churn-output-bucket-us-east-1
churn-output-bucket-us-east-1
arn:aws:iam::852619674999:role/service-role/AmazonSageMaker-ExecutionRole-20220427T124311


# Input Data Location Parameters

In [3]:

# Default location for the datasets
train_data_uri = build_parameters["train_data"]
test_data_uri = build_parameters["test_data"]
evaluation_data_uri = build_parameters["evaluation_data"]
feature_selection_file_uri = build_parameters["feature_selection"]


# Parametrizing Data paths
from sagemaker.workflow.parameters import ParameterInteger, ParameterString
train_data = ParameterString(name="TrainData", default_value = train_data_uri)
test_data = ParameterString(name="TestData", default_value = test_data_uri)
evaluation_data = ParameterString(name="EvaluationData", default_value = evaluation_data_uri)
feature_selection_file = ParameterString(name = "FeatureSelectionFile", default_value = feature_selection_file_uri)


# Preprocessing Training Data

#### 2.1 Loading preprocessing config.json file.

In [4]:
local_preprocessing_path = os.path.join("Pipeline_Component_Codes","Training","1_Preprocessing")
with open(os.path.join(local_preprocessing_path, "config.json")) as file:
    processing_build_parameters = json.load(file)

#### 2.2 Making parameter for processing machine type

In [5]:
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value=processing_build_parameters["machine_type"]
)


#### 2.3 Building the processor

In [6]:

if processing_build_parameters["processing_type"] == "sklearn":
    from sagemaker.sklearn.processing import SKLearnProcessor
    processor = SKLearnProcessor(
        framework_version = processing_build_parameters["framework_version"],
        instance_type = processing_build_parameters["machine_type"],
        instance_count = processing_build_parameters["machine_count"],
        base_job_name = f"{build_parameters['usecase']}-preprocessing",
        role=role
    )
elif processing_build_parameters["processing_type"] == "custom":
    from sagemaker.processor import Processor
    processor = Processor(
        image_uri = processing_build_parameters["image_uri"],
        instance_type = processing_build_parameters["machine_type"],
        instance_count = processing_build_parameters["machine_count"],
        base_job_name = f"{build_parameters['usecase']}-preprocessing",
        role=role
    )

#### 2.4 Building preprocessing step

In [7]:

from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name = "preprocessing_full_data",
    description = "Data preprocessing and splitting into train and test set",
    processor=processor,
    inputs=[
        ProcessingInput(source = train_data, destination="/opt/ml/processing/input/data"),  
        ProcessingInput(source=feature_selection_file, destination="/opt/ml/processing/input/feature_selection")
    ],
    outputs=[
        # Train
        ProcessingOutput(output_name = "train", source="/opt/ml/processing/train", 
                         destination = sagemaker_session.default_bucket
#                          destination = sagemaker.workflow.functions.Join(on='/', values = [processing_output_path, "data"])
                        ),
        # Test
        ProcessingOutput(output_name = "test", source="/opt/ml/processing/test", 
                         destination = sagemaker_session.default_bucket
#                          destination = sagemaker.workflow.functions.Join(on='/', values = [processing_output_path, "data"])
                        ),
        # Logs
        ProcessingOutput(output_name = "logs", source="/opt/ml/processing/logss", 
                         destination = sagemaker_session.default_bucket
#                          destination = sagemaker.workflow.functions.Join(on='/', values = [processing_output_path, "logs"])
                        ),
    ],
#     code="SageMaker_Pipeline_Component_Codes/Training/Training_Preprocessing.py",
    code=os.path.join(local_preprocessing_path, processing_build_parameters["entry_point"]),
    job_arguments = ["--train_data_location", "/opt/ml/processing/input/data", 
                     "--feature_selection_file_location", "/opt/ml/processing/input/feature_selection/Feature_Selection.csv", 
                     "--target_column", "Churn",
                     "--preprocessed_train_data_location", "/opt/ml/processing/train", 
                     "--preprocessed_test_data_location", "/opt/ml/processing/test", 
                     "--log_location", "/opt/ml/processing/logss"
                    ]
)


# 3. Hyperparameter Tuning

#### 3.1 Getting models from Local Path

In [8]:

local_models_path = os.path.join("Pipeline_Component_Codes", "Training", "2_Models_HPTune")
models = []
for directory in os.listdir(local_models_path):
    if '.' not in directory: # Avoiding .ipynb_checkpoints
        models.append(directory)

models = models[:-1]
print(models)

['Decision_Tree', 'Logistic_Regression']


#### 3.2 Reading config.json Files

In [9]:
model_details = {}
for model in models:
    with open(os.path.join(local_models_path, model, "config.json")) as file:
        model_build_parameters = json.load(file)
    model_details[model] = model_build_parameters

# print(model_details)

#### 3.2 Input Parameters

In [10]:
# Machine types
training_instances = []
for model in models:
    training_instance_type = ParameterString(
        name=f"{model}_InstanceType",
        default_value="ml.m5.xlarge"
    )
    training_instances.append(training_instance_type)

# Objective metric
# objective_metric_name = ParameterString(name = "ObjectiveMetric", default_value = build_parameters["objective_metric"])
# metric_definitions = [{"Name": objective_metric_name, "Regex": "accuracy:([0-9\\.]+)"}]

objective_metric_name = "validation:accuracy"
metric_definitions = [{'Name': "validation:accuracy", 'Regex': "accuracy:([0-9\\.]+)"}]

#### 3.3 Creating the Estimators on which Tuning Will Happen

Sagemaker provides us docker conatiners for all the popular algorithms like they have scikit learn image for all the Scikit learn models, they have XGBoost image and they also have deep learning images as well. Here for the demo purpose we have used only three models, Logistic Regression, Decision Tree and XGBoost. So our need was not to build our own image. If any other Python library is needed we can mention those in **requirements.txt** file. We do not have to do anything more, pipeline will automatically install those in respective containers. 

In [11]:
from sagemaker.sklearn import SKLearn
from sagemaker.xgboost.estimator import XGBoost

from sagemaker.tuner import ContinuousParameter, IntegerParameter, CategoricalParameter, HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep

from sagemaker.workflow import functions



tuning_steps = []
index = -1
for model in models:
    index = index + 1
    current_model_details = model_details[model]
    
    # Creating the Estimators on which Tuning Will Happen
    if current_model_details['model_type'] == 'sklearn_model':
        estimator = SKLearn(
                            source_dir = os.path.join(local_models_path, model),
                            entry_point = current_model_details["entry_point"], 
                            instance_type = training_instances[index], 
                            instance_count = 1,
                            framework_version = current_model_details["framework_version"], 
                            role = role,
                            # output_path = f"{hptune_training_output_path}/{model_details['model_name']}",
                            # output_path = functions.Join(on = '/', values = [pipeline_session.default_bucket, "churn_training"])
                            )
    elif current_model_details['model_type'] == 'xgboost_model':
        estimator = XGBoost(
                            source_dir = os.path.join(local_models_path, model),
                            entry_point = current_model_details["entry_point"],
                            instance_type = training_instances[index],
                            instance_count = 1,
                            framework_version = current_model_details["framework_version"],
                            role=role,
                            # output_path = f"{hptune_training_output_path}/{model_details['model_name']}",
                            # output_path = functions.Join(on = '/', values = [pipeline_session.default_bucket, "churn_training"])
                            )
    
    
    # Getting the hyperparameters
    hyperparameters = current_model_details["hyperparameters"]
    
    hyperparameter_ranges = {}
    for hyperparameter in hyperparameters:
        if hyperparameters[hyperparameter]["type"] == "continuous":
            hyperparameter_ranges[hyperparameter] = ContinuousParameter(min_value = hyperparameters[hyperparameter]["min_value"],
                                                                        max_value = hyperparameters[hyperparameter]["max_value"],
                                                                        scaling_type = hyperparameters[hyperparameter]["scaling_type"])
        elif hyperparameters[hyperparameter]["type"] == "categorical":
            hyperparameter_ranges[hyperparameter] = CategoricalParameter(hyperparameters[hyperparameter]["values"])
        elif hyperparameters[hyperparameter]["type"] == "integer":
            hyperparameter_ranges[hyperparameter] = IntegerParameter(min_value = hyperparameters[hyperparameter]["min_value"],
                                                                     max_value = hyperparameters[hyperparameter]["max_value"])
    
    
    
    # Making the hyperparameter tuner
    tuner = HyperparameterTuner(
        estimator = estimator,
        objective_metric_name = objective_metric_name,
        hyperparameter_ranges = hyperparameter_ranges,
        metric_definitions = metric_definitions,
        max_jobs=1,
        max_parallel_jobs=1,
        strategy = current_model_details["tuning_strategy"],
        base_tuning_job_name = current_model_details["model_name"]
        )
    
    
    # Building the tuning step
    step_tuning = TuningStep(
        name = f"hptuning_{current_model_details['model_name']}",
        tuner = tuner,
        inputs={
            "train": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "test": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
                content_type="text/csv",
            ),
        }
    )
    
    tuning_steps.append(step_tuning)
        

instance_type is a PipelineVariable (<class 'sagemaker.workflow.parameters.ParameterString'>). Its interpreted value in execution time should not be of GPU types since GPU training is not supported for Scikit-Learn.
The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
instance_type is a PipelineVariable (<class 'sagemaker.workflow.parameters.ParameterString'>). Its interpreted value in execution time should not be of GPU types since GPU training is not supported for Scikit-Learn.
The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure 

# 4. Getting the best model from each hyperparameter tuning job

Follow this link to get example of how to write lambda step https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-pipelines/tabular/lambda-step/sagemaker-pipelines-lambda-step.ipynb

#### Building the Lambda Function

In [12]:
from sagemaker.lambda_helper import Lambda


func = Lambda(
    function_name = "get_best_model_from_hptune_job",
    execution_role_arn="arn:aws:iam::852619674999:role/role_given_to_lambda",
#     execution_role_arn = role,
#     execution_role_arn = lambda_role,
    script = os.path.join("Pipeline_Component_Codes", "Training", "3_Model_Selection", "main.py"),
    handler="main.main",
)


#### Building the Lambdastep
See this link to get the idea on how tuning step name is being fetched (tuning_steps[i].properties.HyperParameterTuningJobName): https://boto3.amazonaws.com/v1/documentation/api/1.9.46/reference/services/sagemaker.html#SageMaker.Client.describe_hyper_parameter_tuning_job 
In the same way other properties can also be accessed.

In [13]:
from sagemaker.workflow.lambda_step import LambdaOutput, LambdaStep, LambdaOutputTypeEnum

best_model_steps = []
for i in range(len(models)):
    # Building the outputs
    output_param_1 = LambdaOutput(output_name=f"best_{models[i]}_model_location", output_type=LambdaOutputTypeEnum.String)
    output_param_2 = LambdaOutput(output_name=f"best_{models[i]}_metric_value", output_type=LambdaOutputTypeEnum.String)
    
    # Building the Lambdastep
    step_deploy_lambda = LambdaStep(
        name=f"get_best_{models[i]}_model",
        lambda_func=func,
        inputs={
            "tuning_job_name": tuning_steps[i].properties.HyperParameterTuningJobName
        },
        outputs=[output_param_1, output_param_2]
    )
    
    best_model_steps.append(step_deploy_lambda)

In [13]:

current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
model_name = "demo-lambda-model" + current_time
endpoint_config_name = "demo-lambda-deploy-endpoint-config-" + current_time
endpoint_name = "demo-lambda-deploy-endpoint-" + current_time

function_name = "sagemaker-lambda-step-endpoint-deploy-" + current_time

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="code/lambda_helper.py",
    handler="lambda_helper.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    name="LambdaStep",
    lambda_func=func,
    inputs={
        "model_name": step_create_model.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

NameError: name 'time' is not defined

In [15]:
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
inputs = CreateModelInput(
    instance_type=build_parameters["evaluation_instance_type"],
    # accelerator_type="ml.eia1.medium",
)
from sagemaker.workflow.steps import CreateModelStep

create_best_model_steps = []
entry_point='SageMaker_Pipeline_Component_Codes/Training/Evaluation.py',
for i in range(n_models):
    tuning_step_best_model = Model(image_uri = tuning_steps[i].tuner.estimator.image_uri, 
                                   source_dir = f"s3://{pipeline_input_bucket}/codes/evaluation.tar.gz",
                                   # source_dir = build_parameters["single_model_evluation_source_dir"],
                                   entry_point = build_parameters["single_model_evluation_entry_point"],
                                   model_data = sagemaker.workflow.functions.Join(on='/', values=[hptune_training_output_path, tuning_steps[i].name[9:], tuning_steps[i].properties.BestTrainingJob.TrainingJobName, "output/model.tar.gz"]), 
                                   role = role,
                                   sagemaker_session = sagemaker_session
                                  )
    
    step_create_best_model = CreateModelStep(
        name = f"Getting-Best-{tuning_steps[i].name[9:]}-Model",
        model = tuning_step_best_model,
        inputs = inputs
    )
    
    create_best_model_steps.append(step_create_best_model)

### Evaluating the best models from each hyperparameter tuning job

In [16]:
from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.properties import PropertyFile

evaluation_steps = []

for i in range(n_models):
    transformer_dt = Transformer(
        model_name = create_best_model_steps[i].properties.ModelName,
        instance_type = build_parameters["evaluation_instance_type"],
        instance_count=1,
        output_path=f"{hptune_training_output_path}/{tuning_steps[i].name[9:]}/BestModel",
        base_transform_job_name = f"{usecase}-evaluation-{tuning_steps[i].name[9:]}",
        env = {"MODELS3LOCATION":create_best_model_steps[i].properties.PrimaryContainer.ModelDataUrl, 
               "MODELNAME":build_parameters["model_specifications"][f"model{i}"]["model_name"]}
    )
    evaluation_step = TransformStep(
        name=f"Evaluating-Best-{tuning_steps[i].name[9:]}-Model",
        transformer=transformer_dt,
        inputs=TransformInput(data=step_process_evaluation.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, 
                              # data_type = "text/csv"
                             )
    )
    evaluation_steps.append(evaluation_step)

### Getting the best model based on model performance metric on evaluation data

In [17]:
inputs = []
for i in range(n_models):
    inputs.append(ProcessingInput(sagemaker.workflow.functions.Join(on='/', values=[evaluation_steps[i].properties.TransformOutput.S3OutputPath, "evaluation.csv.out"]), destination=f"/opt/ml/processing/input/model{i}"))
# inputs = inputs + [ProcessingInput(sagemaker.workflow.functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output", "Model_Performance_Metrics.csv"]), destination=f"/opt/ml/processing/metrics")]
inputs = inputs + [ProcessingInput(source = sagemaker.workflow.functions.Join(on='/', values=["s3:/", pipeline_input_bucket, "codes", "preprocessing_requirements.txt"]), destination = "/opt/ml/processing/input/requirements")]
# inputs = inputs + [ProcessingInput(source = sagemaker.workflow.functions.Join(on='/', values=["s3:/", pipeline_input_bucket, "codes", "preprocessing_requirements.txt"]), destination = "/opt/ml/processing/input/")]

In [18]:
from sagemaker.workflow.properties import PropertyFile

property_file = PropertyFile(
    name="property_file",
    output_name="property_file",
    path="property_file.json"
)

step_get_best_model = ProcessingStep(
    name = "Getting-Best-Model",
    description = "Picking the best model based on the metric value calculated using evaluation data",
    processor = sklearn_processor,
    inputs=inputs,
    outputs=[
        ProcessingOutput(output_name="final_model", source = "/opt/ml/processing/final_model", destination = evaluation_output_path),
        ProcessingOutput(output_name="logs", source = "/opt/ml/processing/logs", destination = evaluation_output_path),
        ProcessingOutput(output_name="Metrics", source = "/opt/ml/processing/metrics_folder", 
                         destination = functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output"])
                        ),
                         # f"s3://{pipeline_output_bucket}/Training_Pipeline_Output/")
        ProcessingOutput(output_name="Feature_Importance", source = "/opt/ml/processing/feature_importance", 
                         destination = functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output"])
                        ),
        ProcessingOutput(output_name="Confusion_Matrix", source = "/opt/ml/processing/confusion_matrix", 
                         destination = functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output"])
                        ),
        ProcessingOutput(output_name="Combined_Dashboard_Data", source = "/opt/ml/processing/Combined", 
                         destination = functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output"])
                        ),
        ProcessingOutput(output_name="property_file", source = "/opt/ml/processing/evaluation", destination = evaluation_output_path)
    ],
    # code="SageMaker_Pipeline_Component_Codes/Training/Final_Model_Selection.py",
    code = f"s3://{pipeline_input_bucket}/codes/{build_parameters['get_best_model_code_file_name']}",
    # depends_on = [step_dt_evaluation, step_lr_evaluation],
    depends_on = evaluation_steps,
    job_arguments = ["--input_folder", "/opt/ml/processing/input", "--final_model_location", "/opt/ml/processing/final_model", 
                     "--logs_location", "/opt/ml/processing/logs", 
                     # "--model_metric_input_location", "/opt/ml/processing/metrics", 
                     "--model_metric_input_location", sagemaker.workflow.functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output", "Model_Performance_Metrics.csv"]),
                     "--model_metric_output_location", "/opt/ml/processing/metrics_folder", "--objective_metric", objective_metric_name, 
                     "--property_file_location", "/opt/ml/processing/evaluation", 
                     "--feature_importance_input_file_location", sagemaker.workflow.functions.Join(on='/', values=["s3:/", pipeline_output_bucket, "Training_Pipeline_Output", "Feature_Importance.csv"]),
                     "--feature_importance_output_file_location", "/opt/ml/processing/feature_importance"
                    ],
    property_files=[property_file]
)

### Register best model in SageMaker model registry

In [19]:
from sagemaker.workflow.step_collections import RegisterModel

register_best_model_steps = []

for i in range(n_models):
    model_details = build_parameters["model_specifications"][f"model{i}"]
    if model_details["model_type"] == 'sklearn_model':
        estimator = SKLearn(entry_point = "", 
                            
                            instance_type = model_details["instance_type"],
                            framework_version = '0.20.0', 
                            image_uri = sklearn_image_uri,
                            
                            role = role
                            )
        register_best_model_step = RegisterModel(name=f"RegisterBest{model_details['model_name']}Model", 
                              estimator = estimator, 
                              # model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_path),
                              model_data=sagemaker.workflow.functions.Join(on='/', values=[step_get_best_model.properties.ProcessingOutputConfig.Outputs["final_model"].S3Output.S3Uri, "model.tar.gz"]),
                              content_types=["text/csv"],
                              response_types=["text/csv"],
                              inference_instances=[model_details["instance_type"]],
                              transform_instances=[model_details["instance_type"]],
                              model_package_group_name = build_parameters["model_package_group_name"],
                              image_uri = sklearn_image_uri,
                              # approval_status="Approved",
                              role=role,
                              depends_on = []
                             )
        register_best_model_steps.append(register_best_model_step)

In [ ]:
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


condition_steps = []
for i in range(n_models):
    model_details = build_parameters["model_specifications"][f"model{i}"]
    condition_equal = ConditionEquals(left = JsonGet(step_name=step_get_best_model.name, 
                                                   property_file=property_file, 
                                                   json_path="best_model_name"),
                                      right = model_details["model_name"]
                                     )
    step_cond = ConditionStep(
        name=f"Is-{model_details['model_name']}-Best-Model",
        conditions=[condition_equal],
        if_steps = [register_best_model_steps[i]],
        )
    condition_steps.append(step_cond)

### Model Given

In [ ]:
# if build_parameters["given_model_type"] == "sklearn":
#     estimator = SKLearn(entry_point = "", 
                        
#                         instance_type = build_parameters["scoring_instance_type"],
#                         framework_version = '0.20.0', 
#                         image_uri = sklearn_image_uri,
                        
#                         role = role
#                         )
    
#     register_given_model_step = RegisterModel(name=f"RegisterGivenModel", 
#                                              estimator = estimator, 
#                                              # model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_path),
#                                              model_data=build_parameters["given_model_path"],
#                                              content_types=["text/csv"],
#                                              response_types=["text/csv"],
#                                              inference_instances=[build_parameters["scoring_instance_type"]],
#                                              transform_instances=[build_parameters["scoring_instance_type"]],
#                                              model_package_group_name = build_parameters["model_package_group_name"],
#                                              image_uri = sklearn_image_uri,
#                                              approval_status="Approved",
#                                              role=role,
#                                              depends_on = []
#                                             )

In [ ]:
# model_given_condition = ConditionEquals(left = build_parameters["model_given"],
#                                         right = "No"
#                                        )
# step_model_given_cond = ConditionStep(
#     name=f"Is-Model-Given",
#     conditions=[model_given_condition],
#     # if_steps = [register_given_model_step],
#     if_steps = [step_process] + tuning_steps + [step_process_evaluation] + create_best_model_steps + evaluation_steps + [step_get_best_model] + condition_steps,
#     else_steps = [register_given_model_step]
# )

### Arranging the steps inside pipeline

In [25]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"{usecase}-training"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        train_data,
        test_data,
        evaluation_data,
        feature_selection_file,
        pipeline_output_bucket,
        #model_given,
        #model_path,
#         pipeline_output_path,
        # processing_instance_count,
        objective_metric_name,
        # processing_code_location,
        #training_instance_type,
        #evaluation_instance_type,
        processing_instance_type,
        training_instance_type
    ],
#     steps=[step_process, step_process_evaluation, step_tuning_dt, step_tuning_lr, step_create_best_dt_model, step_cond],
#     steps=[step_process_evaluation, step_process, step_tuning_dt, step_tuning_lr, step_create_best_dt_model, step_create_best_lr_model, step_dt_evaluation, step_lr_evaluation, step_cond]
#     steps=[step_process, step_tuning_dt, step_process_evaluation, step_tuning_lr, step_create_best_dt_model, step_create_best_lr_model, step_dt_evaluation, step_lr_evaluation, step_get_best_model, step_register_best_model]
#     steps = [step_cond]
    steps = [step_process] + tuning_steps + [step_process_evaluation] + create_best_model_steps + evaluation_steps + [step_get_best_model] + condition_steps
#     steps = [step_model_given_cond]
)

In [14]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"{build_parameters['usecase']}-training"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        train_data,
        test_data,
        evaluation_data,
        feature_selection_file,
        #model_given,
        #model_path,
#         pipeline_output_path,
        # processing_instance_count,
#         objective_metric_name,
        # processing_code_location,
        #training_instance_type,
        #evaluation_instance_type,
        processing_instance_type,
#         training_instance_type
    ] + training_instances + [objective_metric_name],
#     steps=[step_process, step_process_evaluation, step_tuning_dt, step_tuning_lr, step_create_best_dt_model, step_cond],
#     steps=[step_process_evaluation, step_process, step_tuning_dt, step_tuning_lr, step_create_best_dt_model, step_create_best_lr_model, step_dt_evaluation, step_lr_evaluation, step_cond]
#     steps=[step_process, step_tuning_dt, step_process_evaluation, step_tuning_lr, step_create_best_dt_model, step_create_best_lr_model, step_dt_evaluation, step_lr_evaluation, step_get_best_model, step_register_best_model]
#     steps = [step_cond]
    steps = [step_process] + tuning_steps + best_model_steps,
    sagemaker_session = pipeline_session
)

### Uploading the pipeline

In [15]:
pipeline.upsert(role_arn=role)
# execution = pipeline.start()

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:852619674999:pipeline/churn-training',
 'ResponseMetadata': {'RequestId': 'be5f03eb-164a-43b1-ad0a-f7f62f3b84dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'be5f03eb-164a-43b1-ad0a-f7f62f3b84dc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Thu, 04 May 2023 11:33:48 GMT'},
  'RetryAttempts': 0}}

In [19]:
# execution = pipeline.start()